In [1]:
# workaround via specifying an invalid value first
%config Application.log_level='WORKAROUND'
# => fails, necessary on Fedora 27, ipython3 6.2.1
%config Application.log_level='DEBUG'
import logging
logging.getLogger().setLevel(logging.DEBUG)
log = logging.getLogger()
log.debug('Test debug')

ERROR:root:The 'log_level' trait of an IPKernelApp instance must be any of (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL'), but a value of 'WORKAROUND' <class 'str'> was specified.
DEBUG:root:Test debug


In [2]:
transform_id = 6
run_id = 1

In [3]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = SessionHelper().session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            dataset=db_transform.transformation_template.name)


DEBUG:git.cmd:Popen(['git', 'version'], cwd=/host/core/transforms, universal_newlines=False, shell=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/host/core/transforms, universal_newlines=False, shell=None)


2019-07-18 15:52:55,955 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-07-18 15:52:55,976 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-07-18 15:52:56,005 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-07-18 15:52:56,007 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-07-18 15:52:56,011 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-07-18 15:52:56,013 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-07-18 15:52:56,017 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-07-18 15:52:56,018 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-07-18 15:52:5

# CORE Cartridge Notebook :: symphony_health_association_refinement
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [4]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
        ## YOUR properties go here!!
        ingest_source_transform: str = db_transform.variables.ingest_source_transform # The name of the dataset to pull from
        ingest_source_file_prefix: str = db_transform.variables.ingest_source_file_prefix # If from initial ingest, the file prefix name
        transaction_date: str = db_transform.variables.transaction_date
        pharmacy_code: str = db_transform.variables.pharmacy_code
        pharmacy_npi: str = db_transform.variables.pharmacy_npi
        pharmacy_hin: str = db_transform.variables.pharmacy_hin
        pharmacy_name: str = db_transform.variables.pharmacy_name
        pharmacy_ncpdp: str = db_transform.variables.pharmacy_ncpdp
        pharmacy_address_1: str = db_transform.variables.pharmacy_address_1
        pharmacy_address_2: str = db_transform.variables.pharmacy_address_2
        pharmacy_city: str = db_transform.variables.pharmacy_city
        pharmacy_state: str = db_transform.variables.pharmacy_state
        pharmacy_zip: str = db_transform.variables.pharmacy_zip
        transaction_type: str = db_transform.variables.transaction_type
        pharmacy_transaction_id: str = db_transform.variables.pharmacy_transaction_id
        transaction_sequence: str = db_transform.variables.transaction_sequence
        referral_source: str = db_transform.variables.referral_source
        referral_date: str = db_transform.variables.referral_date
        longitudinal_patient_id: str = db_transform.variables.longitudinal_patient_id
        pharmacy_patient_id: str = db_transform.variables.pharmacy_patient_id
        patient_dob: str = db_transform.variables.patient_dob
        hub_patient_id: str = db_transform.variables.hub_patient_id
        bridge_patient: str = db_transform.variables.bridge_patient
        hub_patient: str = db_transform.variables.hub_patient
        patient_gender: str = db_transform.variables.patient_gender
        dx_1: str = db_transform.variables.dx_1
        dx_2: str = db_transform.variables.dx_2
        status_date: str = db_transform.variables.status_date
        status: str = db_transform.variables.status
        substatus: str = db_transform.variables.substatus
        customer_status: str = db_transform.variables.customer_status
        customer_substatus: str = db_transform.variables.customer_substatus
        customer_status_description: str = db_transform.variables.customer_status_description
        hcp_last_name: str = db_transform.variables.hcp_last_name
        hcp_first_name: str = db_transform.variables.hcp_first_name
        hcp_address_1: str = db_transform.variables.hcp_address_1
        hcp_address_2: str = db_transform.variables.hcp_address_2
        hcp_city: str = db_transform.variables.hcp_city
        hcp_state: str = db_transform.variables.hcp_state
        hcp_zip: str = db_transform.variables.hcp_zip
        hcp_phone: str = db_transform.variables.hcp_phone
        hcp_specialty: str = db_transform.variables.hcp_specialty
        hcp_npi: str = db_transform.variables.hcp_npi
        hcp_dea_number: str = db_transform.variables.hcp_dea_number
        hcp_facility: str = db_transform.variables.hcp_facility
        rx_date: str = db_transform.variables.rx_date
        rx_number: str = db_transform.variables.rx_number
        rx_fills: str = db_transform.variables.rx_fills
        rx_fill_number: str = db_transform.variables.rx_fill_number
        rx_refills_remaining: str = db_transform.variables.rx_refills_remaining
        prev_dispensed: str = db_transform.variables.prev_dispensed
        ndc: str = db_transform.variables.ndc
        brand: str = db_transform.variables.brand
        medication: str = db_transform.variables.medication
        quantity_dispensed: str = db_transform.variables.quantity_dispensed
        uom_dispensed: str = db_transform.variables.uom_dispensed
        days_supply: str = db_transform.variables.days_supply
        ship_date: str = db_transform.variables.ship_date
        ship_carrier: str = db_transform.variables.ship_carrier
        ship_tracking_id: str = db_transform.variables.ship_tracking_id
        ship_location: str = db_transform.variables.ship_location
        ship_address_1: str = db_transform.variables.ship_address_1
        ship_address_2: str = db_transform.variables.ship_address_2
        ship_city: str = db_transform.variables.ship_city
        ship_state: str = db_transform.variables.ship_state
        ship_zip: str = db_transform.variables.ship_zip
        has_medical_coverage_flag: str = db_transform.variables.has_medical_coverage_flag
        primary_coverage_type: str = db_transform.variables.primary_coverage_type
        primary_payer: str = db_transform.variables.primary_payer
        primary_payer_type: str = db_transform.variables.primary_payer_type
        primary_payer_subtype: str = db_transform.variables.primary_payer_subtype
        primary_payer_group: str = db_transform.variables.primary_payer_group
        primary_payer_bin: str = db_transform.variables.primary_payer_bin
        primary_payer_iin: str = db_transform.variables.primary_payer_iin
        primary_payer_pcn: str = db_transform.variables.primary_payer_pcn
        primary_plan: str = db_transform.variables.primary_plan
        primary_plan_type: str = db_transform.variables.primary_plan_type
        secondary_coverage_type: str = db_transform.variables.secondary_coverage_type
        secondary_payer: str = db_transform.variables.secondary_payer
        secondary_payer_type: str = db_transform.variables.secondary_payer_type
        secondary_payer_subtype: str = db_transform.variables.secondary_payer_subtype
        secondary_payer_group: str = db_transform.variables.secondary_payer_group
        secondary_payer_bin: str = db_transform.variables.secondary_payer_bin
        secondary_payer_iin: str = db_transform.variables.secondary_payer_iin
        secondary_payer_pcn: str = db_transform.variables.secondary_payer_pcn
        secondary_plan: str = db_transform.variables.secondary_plan
        secondary_plan_type: str = db_transform.variables.secondary_plan_type
        primary_plan_paid: str = db_transform.variables.primary_plan_paid
        secondary_plan_paid: str = db_transform.variables.secondary_plan_paid
        primary_copay: str = db_transform.variables.primary_copay
        primary_coins: str = db_transform.variables.primary_coins
        primary_deductible: str = db_transform.variables.primary_deductible
        primary_patient_responsibility: str = db_transform.variables.primary_patient_responsibility
        secondary_copay: str = db_transform.variables.secondary_copay
        secondary_coins: str = db_transform.variables.secondary_coins
        secondary_deductible: str = db_transform.variables.secondary_deductible
        secondary_patient_responsibility: str = db_transform.variables.secondary_patient_responsibility
        copay_as_amount: str = db_transform.variables.copay_as_amount
        other_payer_amount: str = db_transform.variables.other_payer_amount
        transfer_pharmacy: str = db_transform.variables.transfer_pharmacy
        bridge_quantity_dispensed: str = db_transform.variables.bridge_quantity_dispensed
        prior_therapy_name: str = db_transform.variables.prior_therapy_name
        pharmacy_parent_name: str = db_transform.variables.pharmacy_parent_name
        hcp_state_license_number: str = db_transform.variables.hcp_state_license_number
        patient_state: str = db_transform.variables.patient_state
        dose_exchange_flag: str = db_transform.variables.dose_exchange_flag
        dose_exchange_count: str = db_transform.variables.dose_exchange_count
        dose_titration_quantity: str = db_transform.variables.dose_titration_quantity
        dose_titration_count: str = db_transform.variables.dose_titration_count
        oxygen_flag: str = db_transform.variables.oxygen_flag
        patient_zip: str = db_transform.variables.patient_zip
        secondary_payer_flag: str = db_transform.variables.secondary_payer_flag
        restatement_flag: str = db_transform.variables.restatement_flag
        aggregator_transaction_id: str = db_transform.variables.aggregator_transaction_id
        primary_pbm_name: str = db_transform.variables.primary_pbm_name
        referral_number: str = db_transform.variables.referral_number
        hcp_middle_name: str = db_transform.variables.hcp_middle_name
        hcp_suffix: str = db_transform.variables.hcp_suffix
        pharmacy_dea_number: str = db_transform.variables.pharmacy_dea_number
        primary_prior_auth_required_flag: str = db_transform.variables.primary_prior_auth_required_flag
        primary_prior_auth_expiration_date: str = db_transform.variables.primary_prior_auth_expiration_date
        patient_consent_date: str = db_transform.variables.patient_consent_date
        primary_cost_type: str = db_transform.variables.primary_cost_type
        primary_cost_amount: str = db_transform.variables.primary_cost_amount
        patient_support_1: str = db_transform.variables.patient_support_1
        patient_support_2: str = db_transform.variables.patient_support_2
        patient_oop_program_name: str = db_transform.variables.patient_oop_program_name
        bridge_quantity_dispensed_2: str = db_transform.variables.bridge_quantity_dispensed_2
        enroll_received_date: str = db_transform.variables.enroll_received_date
        fitness_for_duty_request_flag: str = db_transform.variables.fitness_for_duty_request_flag
        fitness_for_duty_ship_date: str = db_transform.variables.fitness_for_duty_ship_date
        triage_date: str = db_transform.variables.triage_date
        dose_count: str = db_transform.variables.dose_count

In [5]:
from core.logging import get_logger

In [6]:
transform = Transform()
logger = get_logger(f"core.transforms.{transform.state}.{transform.name}")

In [48]:
# The data model is hardcoded as a constant that will require a PR to edit for security
column_renames = {
    transform.transaction_date : 'transaction_date',
    transform.pharmacy_code : 'pharmacy_code',
    transform.pharmacy_npi : 'pharmacy_npi',
    transform.pharmacy_hin : 'pharmacy_hin',
    transform.pharmacy_name : 'pharmacy_name',
    transform.pharmacy_ncpdp : 'pharmacy_ncpdp',
    transform.pharmacy_address_1 : 'pharmacy_address_1',
    transform.pharmacy_address_2 : 'pharmacy_address_2',
    transform.pharmacy_city : 'pharmacy_city',
    transform.pharmacy_state : 'pharmacy_state',
    transform.pharmacy_zip : 'pharmacy_zip',
    transform.transaction_type : 'transaction_type',
    transform.pharmacy_transaction_id : 'pharmacy_transaction_id',
    transform.transaction_sequence : 'transaction_sequence',
    transform.referral_source : 'referral_source',
    transform.referral_date : 'referral_date',
    transform.longitudinal_patient_id : 'longitudinal_patient_id',
    transform.pharmacy_patient_id : 'pharmacy_patient_id',
    transform.patient_dob : 'patient_dob',
    transform.hub_patient_id : 'hub_patient_id',
    transform.bridge_patient : 'bridge_patient',
    transform.hub_patient : 'hub_patient',
    transform.patient_gender : 'patient_gender',
    transform.dx_1 : 'dx_1',
    transform.dx_2 : 'dx_2',
    transform.status_date : 'status_date',
    transform.status : 'status',
    transform.substatus : 'substatus',
    transform.customer_status : 'customer_status',
    transform.customer_substatus : 'customer_substatus',
    transform.customer_status_description : 'customer_status_description',
    transform.hcp_last_name : 'hcp_last_name',
    transform.hcp_first_name : 'hcp_first_name',
    transform.hcp_address_1 : 'hcp_address_1',
    transform.hcp_address_2 : 'hcp_address_2',
    transform.hcp_city : 'hcp_city',
    transform.hcp_state : 'hcp_state',
    transform.hcp_zip : 'hcp_zip',
    transform.hcp_phone : 'hcp_phone',
    transform.hcp_specialty : 'hcp_specialty',
    transform.hcp_npi : 'hcp_npi',
    transform.hcp_dea_number : 'hcp_dea_number',
    transform.hcp_facility : 'hcp_facility',
    transform.rx_date : 'rx_date',
    transform.rx_number : 'rx_number',
    transform.rx_fills : 'rx_fills',
    transform.rx_fill_number : 'rx_fill_number',
    transform.rx_refills_remaining : 'rx_refills_remaining',
    transform.prev_dispensed : 'prev_dispensed',
    transform.ndc : 'ndc',
    transform.brand : 'brand',
    transform.medication : 'medication',
    transform.quantity_dispensed : 'quantity_dispensed',
    transform.uom_dispensed : 'uom_dispensed',
    transform.days_supply : 'days_supply',
    transform.ship_date : 'ship_date',
    transform.ship_carrier : 'ship_carrier',
    transform.ship_tracking_id : 'ship_tracking_id',
    transform.ship_location : 'ship_location',
    transform.ship_address_1 : 'ship_address_1',
    transform.ship_address_2 : 'ship_address_2',
    transform.ship_city : 'ship_city',
    transform.ship_state : 'ship_state',
    transform.ship_zip : 'ship_zip',
    transform.has_medical_coverage_flag : 'has_medical_coverage_flag',
    transform.primary_coverage_type : 'primary_coverage_type',
    transform.primary_payer : 'primary_payer',
    transform.primary_payer_type : 'primary_payer_type',
    transform.primary_payer_subtype : 'primary_payer_subtype',
    transform.primary_payer_group : 'primary_payer_group',
    transform.primary_payer_bin : 'primary_payer_bin',
    transform.primary_payer_iin : 'primary_payer_iin',
    transform.primary_payer_pcn : 'primary_payer_pcn',
    transform.primary_plan : 'primary_plan',
    transform.primary_plan_type : 'primary_plan_type',
    transform.secondary_coverage_type : 'secondary_coverage_type',
    transform.secondary_payer : 'secondary_payer',
    transform.secondary_payer_type : 'secondary_payer_type',
    transform.secondary_payer_subtype : 'secondary_payer_subtype',
    transform.secondary_payer_group : 'secondary_payer_group',
    transform.secondary_payer_bin : 'secondary_payer_bin',
    transform.secondary_payer_iin : 'secondary_payer_iin',
    transform.secondary_payer_pcn : 'secondary_payer_pcn',
    transform.secondary_plan : 'secondary_plan',
    transform.secondary_plan_type : 'secondary_plan_type',
    transform.primary_plan_paid : 'primary_plan_paid',
    transform.secondary_plan_paid : 'secondary_plan_paid',
    transform.primary_copay : 'primary_copay',
    transform.primary_coins : 'primary_coins',
    transform.primary_deductible : 'primary_deductible',
    transform.primary_patient_responsibility : 'primary_patient_responsibility',
    transform.secondary_copay : 'secondary_copay',
    transform.secondary_coins : 'secondary_coins',
    transform.secondary_deductible : 'secondary_deductible',
    transform.secondary_patient_responsibility : 'secondary_patient_responsibility',
    transform.copay_as_amount : 'copay_as_amount',
    transform.other_payer_amount : 'other_payer_amount',
    transform.transfer_pharmacy : 'transfer_pharmacy',
    transform.bridge_quantity_dispensed : 'bridge_quantity_dispensed',
    transform.prior_therapy_name : 'prior_therapy_name',
    transform.pharmacy_parent_name : 'pharmacy_parent_name',
    transform.hcp_state_license_number : 'hcp_state_license_number',
    transform.patient_state : 'patient_state',
    transform.dose_exchange_flag : 'dose_exchange_flag',
    transform.dose_exchange_count : 'dose_exchange_count',
    transform.dose_titration_quantity : 'dose_titration_quantity',
    transform.dose_titration_count : 'dose_titration_count',
    transform.oxygen_flag : 'oxygen_flag',
    transform.patient_zip : 'patient_zip',
    transform.secondary_payer_flag : 'secondary_payer_flag',
    transform.restatement_flag : 'restatement_flag',
    transform.aggregator_transaction_id : 'aggregator_transaction_id',
    transform.primary_pbm_name : 'primary_pbm_name',
    transform.referral_number : 'referral_number',
    transform.hcp_middle_name : 'hcp_middle_name',
    transform.hcp_suffix : 'hcp_suffix',
    transform.pharmacy_dea_number : 'pharmacy_dea_number',
    transform.primary_prior_auth_required_flag : 'primary_prior_auth_required_flag',
    transform.primary_prior_auth_expiration_date : 'primary_prior_auth_expiration_date',
    transform.patient_consent_date : 'patient_consent_date',
    transform.primary_cost_type : 'primary_cost_type',
    transform.primary_cost_amount : 'primary_cost_amount',
    transform.patient_support_1 : 'patient_support_1',
    transform.patient_support_2 : 'patient_support_2',
    transform.patient_oop_program_name : 'patient_oop_program_name',
    transform.bridge_quantity_dispensed_2 : 'bridge_quantity_dispensed_2',
    transform.enroll_received_date : 'enroll_received_date',
    transform.fitness_for_duty_request_flag : 'fitness_for_duty_request_flag',
    transform.fitness_for_duty_ship_date : 'fitness_for_duty_ship_date',
    transform.triage_date : 'triage_date',
    transform.dose_count : 'dose_count',
}

# Likewise, the list of required columns is kept as well
required_columns = [
    "pharmacy_code",
    "pharmacy_npi",
    "pharmacy_name",
    "pharmacy_ncpdp",
    "pharmacy_transaction_id",
    "transaction_sequence",
    "pharmacy_patient_id",
    "patient_dob",
    "patient_gender",
    "status_date",
    "status",
    "substatus",
    "hcp_last_name",
    "hcp_first_name",
    "hcp_address_1",
    "hcp_city",
    "hcp_state",
    "hcp_npi",
    "rx_date",
    "ndc",
    "medication",
    "days_supply",
    "has_medical_coverage_flag",
    "primary_coverage_type",
    "primary_payer_type",
    "patient_state",
    "patient_zip"
]

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This transform takes patient status data and maps it to our internal schema.

### Transformation

In [70]:
import pandas as pd
import numpy as np
import boto3
from s3parq import fetch_diff, fetch, publish

In [44]:
# Get dataframes only by file name
# This has to be done through string manipulation on files under the prefix
# TODO: Find a more elegant solution
#    If problematique files get ingested without file name barrier it will never fetch properly

# ingest_prefix = transform.ingest_source_transform + "/" + transform.ingest_source_file_prefix
input_contract = DatasetContract(parent=transform.publish_contract.parent, 
                                child=transform.publish_contract.child,
                                state="ingest",
                                dataset=transform.ingest_source_transform
                                )

ingest_prefix=input_contract.key+"/."+transform.ingest_source_file_prefix
bucket = transform.publish_contract.env

# Need : set of file names
#    Fetching will be iterating over them, fetching with key-less S3 paths

# Get names of all files under key
file_names = set()
files_with_prefix = []
s3_client = boto3.client('s3')
paginator = s3_client.get_paginator('list_objects')
operation_parameters = {'Bucket': bucket,
                        'Prefix': ingest_prefix}
page_iterator = paginator.paginate(**operation_parameters)
for page in page_iterator:
    if not "Contents" in page.keys():
        break

    for item in page['Contents']:
        if item['Key'].endswith('.parquet'):
            files_with_prefix.append(item['Key'])
            
# Remove key
key_len = len(ingest_prefix)

def subtract_key(file):
    # +1 due to the extra slash at the end
    return file[(key_len + 1):]

files_without_prefix = [subtract_key(file) for file in files_with_prefix]

for file_path in files_without_prefix:
    # Split up so other parts can be easily deleted
    unparsed_parts = file_path.split("/")
    
    # Remove parquet names at the end
    del unparsed_parts[-1]

    # Remove partition columns
    for part in unparsed_parts:
        if "=" not in part:
            file_names.add(part)

In [10]:
file_names

{'.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20181001_01_299.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190124_02_266.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190125_01_267.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190128_01_264.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190129_01_263.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190130_01_269.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190131_01_273.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190201_01_278.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190204_01_283.txt',
 '.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190205_01_289.txt',
 '.upload.

# Place your import contracts here
ingest_contract = DatasetContract(branch=BRANCH_NAME,
                                parent=transform.pharmaceutical_company,
                                child=transform.brand,
                                state=transform.state,
                                dataset=ingest_dataset)



In [71]:
def check_required_columns(df:pd.DataFrame):
    logger.debug(f"Expecting the following columns : {list(column_renames.keys())}")
    logger.debug(f"Dataframe has these columns : {list(df.columns)}")
    logger.debug(f"Only the following columns are required : {required_columns}")

    required_columns_preschema = [k for k,v in column_renames.items() if v in required_columns]
    
    missing_columns = set(column_renames.keys()) - set(df.columns)
    missing_required_columns = set(required_columns_preschema) - set(df.columns)

    if missing_required_columns:
        logger.error("Data does not have all required columns.")
        raise ValueError(f"Missing required columns : {missing_required_columns}")
    elif missing_columns:
        for col in missing_columns:
            df[col] = np.nan


In [93]:
%%time
final_dataframe=pd.DataFrame()
run_filter = [{"partition": "__metadata_run_id", "comparison": "==", "values": [run_id]}]

for file_name in file_names:
    logger.debug(f"Retrieving data from path : {ingest_prefix}")
    logger.debug(f"Ingesting data under file name : {file_name} , with run_id : {run_id}")
    
    # Run with parallel as False since its slower if the data is not large
    file_df = fetch(bucket=bucket, key=(ingest_prefix+"/"+file_name), filters=run_filter, parallel=False)
    
    logger.debug(f"File data fetched, fetched dataframe shape : {file_df.shape}")
    
    # Check base requirement fullfillment
    check_required_columns(file_df)
    logger.debug("File meets requirements.")
    
    # First cut down to the necesarry columns in case of accidental extras - Pandas wont catch those
    extra_columns = set(file_df.columns) - set(column_renames.keys())
    file_df = file_df.drop(axis=1,labels=list(extra_columns))

    # Rename based on above created configuration variables
    file_df = file_df.rename(column_renames, axis="columns")
    
    logger.debug("File successfully appended.")
    final_dataframe = final_dataframe.append(file_df)

2019-07-18 18:47:43,434 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping/sitwell/cornballer/ingest/initial_ingest/.upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE
2019-07-18 18:47:43,437 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Ingesting data under file name : .upload.sha_dry_run_test_raw_fetch.INTEGRICHAIN_SUN_ACCREDO_STATUSDISPENSE_20190510_01_748.txt , with run_id : 1
         Rec Date Pharm Code   Pharm NPI transType Pharm Transaction Id  \
0  20190509113616    ACCREDO  1346208949       COM           2742975014   
1  20190509115950    ACCREDO  1043309735       COM   246476442019050908   
2  20190509115950    ACCREDO  1346208949       COM   254705232019050909   
3  20190509030237    ACCREDO  1346208949       COM           2788189112   
4  20190509022006    ACCREDO  1346208949       COM   282165152019050902   

  Trans Seq Ref Source        

2019-07-18 18:47:44,370 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Expecting the following columns : ['Rec Date', 'Pharm Code', 'Pharm NPI', '', 'transType', 'Pharm Transaction Id', 'Trans Seq', 'Ref Source', 'Ref Date', 'Pharmacy ID', 'Pat DOB', 'Program ID', 'PatGender', 'Dx1 Code', 'Dx2 Code', 'Status Date', 'Status Code', 'Sub Status', 'Pres Last Name', 'Pres First Name', 'Pres Addr1', 'Pres Addr2', 'Pres City', 'Pres State', 'Pres Zip', 'Pres Phone', 'Pres NPI', 'Pres DEA', 'Facility Name', 'RxDate', 'RxNumber', 'RxRefills', 'RxFill', 'Refill Remaining', 'prev Disp', 'Rx NDC Number', 'brand', 'Medication', 'Quantity', 'Day Supply', 'Ship Date', 'Ship Carrier', 'shipTracking Num', 'Ship Location', 'Ship Address', 'Ship City', 'Ship State', 'Ship Zip', 'Has Medical', 'Primary CoverageType', 'Primary Payer Name', 'Primary Payer Type', 'Secondary CoverageType', 'Secondary PayerName', 'Secondary PayerType', 'Plan Paid Amt', 'Pat Copay', 'Copay Assist Amount

2019-07-18 18:47:44,831 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Only the following columns are required : ['pharmacy_code', 'pharmacy_npi', 'pharmacy_name', 'pharmacy_ncpdp', 'pharmacy_transaction_id', 'transaction_sequence', 'pharmacy_patient_id', 'patient_dob', 'patient_gender', 'status_date', 'status', 'substatus', 'hcp_last_name', 'hcp_first_name', 'hcp_address_1', 'hcp_city', 'hcp_state', 'hcp_npi', 'rx_date', 'ndc', 'medication', 'days_supply', 'has_medical_coverage_flag', 'primary_coverage_type', 'primary_payer_type', 'patient_state', 'patient_zip']
2019-07-18 18:47:44,835 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File meets requirements.
2019-07-18 18:47:44,840 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File successfully appended.
2019-07-18 18:47:44,846 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping

2019-07-18 18:47:45,794 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Expecting the following columns : ['Rec Date', 'Pharm Code', 'Pharm NPI', '', 'transType', 'Pharm Transaction Id', 'Trans Seq', 'Ref Source', 'Ref Date', 'Pharmacy ID', 'Pat DOB', 'Program ID', 'PatGender', 'Dx1 Code', 'Dx2 Code', 'Status Date', 'Status Code', 'Sub Status', 'Pres Last Name', 'Pres First Name', 'Pres Addr1', 'Pres Addr2', 'Pres City', 'Pres State', 'Pres Zip', 'Pres Phone', 'Pres NPI', 'Pres DEA', 'Facility Name', 'RxDate', 'RxNumber', 'RxRefills', 'RxFill', 'Refill Remaining', 'prev Disp', 'Rx NDC Number', 'brand', 'Medication', 'Quantity', 'Day Supply', 'Ship Date', 'Ship Carrier', 'shipTracking Num', 'Ship Location', 'Ship Address', 'Ship City', 'Ship State', 'Ship Zip', 'Has Medical', 'Primary CoverageType', 'Primary Payer Name', 'Primary Payer Type', 'Secondary CoverageType', 'Secondary PayerName', 'Secondary PayerType', 'Plan Paid Amt', 'Pat Copay', 'Copay Assist Amount

2019-07-18 18:47:46,294 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Only the following columns are required : ['pharmacy_code', 'pharmacy_npi', 'pharmacy_name', 'pharmacy_ncpdp', 'pharmacy_transaction_id', 'transaction_sequence', 'pharmacy_patient_id', 'patient_dob', 'patient_gender', 'status_date', 'status', 'substatus', 'hcp_last_name', 'hcp_first_name', 'hcp_address_1', 'hcp_city', 'hcp_state', 'hcp_npi', 'rx_date', 'ndc', 'medication', 'days_supply', 'has_medical_coverage_flag', 'primary_coverage_type', 'primary_payer_type', 'patient_state', 'patient_zip']
2019-07-18 18:47:46,302 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File meets requirements.
2019-07-18 18:47:46,313 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File successfully appended.
2019-07-18 18:47:46,319 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping

...

2019-07-18 18:48:20,164 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Only the following columns are required : ['pharmacy_code', 'pharmacy_npi', 'pharmacy_name', 'pharmacy_ncpdp', 'pharmacy_transaction_id', 'transaction_sequence', 'pharmacy_patient_id', 'patient_dob', 'patient_gender', 'status_date', 'status', 'substatus', 'hcp_last_name', 'hcp_first_name', 'hcp_address_1', 'hcp_city', 'hcp_state', 'hcp_npi', 'rx_date', 'ndc', 'medication', 'days_supply', 'has_medical_coverage_flag', 'primary_coverage_type', 'primary_payer_type', 'patient_state', 'patient_zip']
2019-07-18 18:48:20,168 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File meets requirements.
2019-07-18 18:48:20,173 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File successfully appended.
2019-07-18 18:48:20,177 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping

2019-07-18 18:48:21,125 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Expecting the following columns : ['Rec Date', 'Pharm Code', 'Pharm NPI', '', 'transType', 'Pharm Transaction Id', 'Trans Seq', 'Ref Source', 'Ref Date', 'Pharmacy ID', 'Pat DOB', 'Program ID', 'PatGender', 'Dx1 Code', 'Dx2 Code', 'Status Date', 'Status Code', 'Sub Status', 'Pres Last Name', 'Pres First Name', 'Pres Addr1', 'Pres Addr2', 'Pres City', 'Pres State', 'Pres Zip', 'Pres Phone', 'Pres NPI', 'Pres DEA', 'Facility Name', 'RxDate', 'RxNumber', 'RxRefills', 'RxFill', 'Refill Remaining', 'prev Disp', 'Rx NDC Number', 'brand', 'Medication', 'Quantity', 'Day Supply', 'Ship Date', 'Ship Carrier', 'shipTracking Num', 'Ship Location', 'Ship Address', 'Ship City', 'Ship State', 'Ship Zip', 'Has Medical', 'Primary CoverageType', 'Primary Payer Name', 'Primary Payer Type', 'Secondary CoverageType', 'Secondary PayerName', 'Secondary PayerType', 'Plan Paid Amt', 'Pat Copay', 'Copay Assist Amount

2019-07-18 18:48:21,650 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Only the following columns are required : ['pharmacy_code', 'pharmacy_npi', 'pharmacy_name', 'pharmacy_ncpdp', 'pharmacy_transaction_id', 'transaction_sequence', 'pharmacy_patient_id', 'patient_dob', 'patient_gender', 'status_date', 'status', 'substatus', 'hcp_last_name', 'hcp_first_name', 'hcp_address_1', 'hcp_city', 'hcp_state', 'hcp_npi', 'rx_date', 'ndc', 'medication', 'days_supply', 'has_medical_coverage_flag', 'primary_coverage_type', 'primary_payer_type', 'patient_state', 'patient_zip']
2019-07-18 18:48:21,654 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File meets requirements.
2019-07-18 18:48:21,657 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File successfully appended.
2019-07-18 18:48:21,661 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping

2019-07-18 18:48:22,582 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Expecting the following columns : ['Rec Date', 'Pharm Code', 'Pharm NPI', '', 'transType', 'Pharm Transaction Id', 'Trans Seq', 'Ref Source', 'Ref Date', 'Pharmacy ID', 'Pat DOB', 'Program ID', 'PatGender', 'Dx1 Code', 'Dx2 Code', 'Status Date', 'Status Code', 'Sub Status', 'Pres Last Name', 'Pres First Name', 'Pres Addr1', 'Pres Addr2', 'Pres City', 'Pres State', 'Pres Zip', 'Pres Phone', 'Pres NPI', 'Pres DEA', 'Facility Name', 'RxDate', 'RxNumber', 'RxRefills', 'RxFill', 'Refill Remaining', 'prev Disp', 'Rx NDC Number', 'brand', 'Medication', 'Quantity', 'Day Supply', 'Ship Date', 'Ship Carrier', 'shipTracking Num', 'Ship Location', 'Ship Address', 'Ship City', 'Ship State', 'Ship Zip', 'Has Medical', 'Primary CoverageType', 'Primary Payer Name', 'Primary Payer Type', 'Secondary CoverageType', 'Secondary PayerName', 'Secondary PayerType', 'Plan Paid Amt', 'Pat Copay', 'Copay Assist Amount

2019-07-18 18:48:23,056 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Only the following columns are required : ['pharmacy_code', 'pharmacy_npi', 'pharmacy_name', 'pharmacy_ncpdp', 'pharmacy_transaction_id', 'transaction_sequence', 'pharmacy_patient_id', 'patient_dob', 'patient_gender', 'status_date', 'status', 'substatus', 'hcp_last_name', 'hcp_first_name', 'hcp_address_1', 'hcp_city', 'hcp_state', 'hcp_npi', 'rx_date', 'ndc', 'medication', 'days_supply', 'has_medical_coverage_flag', 'primary_coverage_type', 'primary_payer_type', 'patient_state', 'patient_zip']
2019-07-18 18:48:23,059 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File meets requirements.
2019-07-18 18:48:23,064 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - File successfully appended.
2019-07-18 18:48:23,070 - core.transforms.ingest.patient_status_ingest_column_mapping - DEBUG - Retrieving data from path : dc-567_patient_status_ingest_mapping

In [89]:
final_dataframe

,transaction_date,pharmacy_code,pharmacy_npi,transaction_type,pharmacy_transaction_id,transaction_sequence,referral_source,referral_date,hub_patient_id,pharmacy_patient_id,...,secondary_coverage_type,secondary_payer,secondary_payer_type,primary_plan_paid,primary_copay,copay_as_amount,other_payer_amount,transfer_pharmacy,dose_count,brand
0,20190509113616,ACCREDO,1346208949,COM,2742975014,0,DIRECT,20181224120000,None,27429750,...,None,None,None,None,None,None,None,None,NaN,NaN
1,20190509115950,ACCREDO,1043309735,COM,246476442019050908,0,DIRECT,20190404120000,None,24647644,...,None,None,None,None,11.61,None,None,None,NaN,NaN
2,20190509115950,ACCREDO,1346208949,COM,254705232019050909,0,DIRECT,20190410120000,None,25470523,...,None,None,None,None,395.00,None,None,None,NaN,NaN
3,20190509030237,ACCREDO,1346208949,COM,2788189112,0,DIRECT,20180928120000,None,27881891,...,None,None,None,None,None,None,None,None,NaN,NaN
4,20190509022006,ACCREDO,1346208949,COM,282165152019050902,0,HUB,20190506120000,None,28216515,...,None,None,None,None,None,None,None,None,NaN,NaN
5,20190509030617,ACCREDO,1346208949,COM,267079842019050903,0,DIRECT,20190509120000,None,26707984,...,None,None,None,None,None,None,None,None,NaN,NaN
6,20190509115950,ACCREDO,1639375066,COM,215812412019050915,0,DIRECT,20181129120000,None,21581241,...,None,None,None,None,11.70,None,None,None,NaN,NaN
7,20190509073136,ACCREDO,1346208949,COM,256545292019050906,0,DIRECT,20190509120000,None,25654529,...,None,None,None,None,None,None,None,None,NaN,NaN
8,20190509062504,ACCREDO,1043309735,COM,246476442019050905,0,DIRECT,20190404120000,None,24647644,...,None,None,None,None,11.61,None,None,None,NaN,NaN
9,20190509115959,ACCREDO,1043309735,COM,259555942019050913,0,DIRECT,20180116120000,None,25955594,...,None,None,None,None,55.00,None,None,None,NaN,NaN


In [90]:
final_dataframe.shape

(1525, 62)

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
# transform.publish_contract.publish(final_dataframe, run_id)